# Workshop: Sentiment Analysis

<div>
<img src="https://lh3.googleusercontent.com/pw/ADCreHdzakFbNdHwBE1ZrwOiNCQibViWOir9DF9Dv4fbZEdWpx4mzFOT_RxkUGLTyDW7fQ0OwEyNQwqllupbvm0WiU0RNuFs-kWx1fTIvjiSkPGE5m64PilOIeApxQLwX_rl-JU7uYT-ROxdppIsJimCeos=w406-h451-s-no-gm?authuser=0" width="390"/> 
</div>




In [ ]:
# !pip install nltk
# !pip install transformers 

## Rule-Based Approaches

- **Lexicon-Based Methods**: Use sentiment lexicons or dictionaries that contain words annotated with their sentiment polarity (positive, negative, neutral).
- **Pattern Matching**: Identify sentiment based on predefined patterns or rules in the text.


In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# nltk.download('stopwords')
# nltk.download('punkt')

In [ ]:
print(stopwords.words('english'))

In [ ]:
text = "I had a good experience with the product. Highly recommended!"

In [ ]:
tokens = word_tokenize(text.lower())

In [ ]:
print(tokens)

In [ ]:
stop_words = set(stopwords.words('english'))

In [ ]:
tokens = [word for word in tokens if word.isalnum() and word not in stop_words]  #alnum = alphanumeric

In [ ]:
print(tokens)

In [ ]:
# Sample positive and negative words
positive_words = set(['good', 'awesome', 'excellent', 'happy', 'positive'])
negative_words = set(['bad', 'terrible', 'poor', 'unhappy', 'negative'])

def rule_based_sentiment_analysis(text):
    # Tokenize the text
    tokens = word_tokenize(text.lower())

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word.isalnum() and word not in stop_words]  #alnum = alphanumeric

    # Calculate sentiment score
    sentiment_score = sum(1 for word in tokens if word in positive_words) - sum(1 for word in tokens if word in negative_words)

    # Classify sentiment
    if sentiment_score > 0:
        return 'Positive'
    elif sentiment_score < 0:
        return 'Negative'
    else:
        return 'Neutral'

# Example usage
text_to_analyze = "I had a good experience with the product. Highly recommended!"
sentiment_result = rule_based_sentiment_analysis(text_to_analyze)
print(f"Sentiment: {sentiment_result}")

## Machine Learning Approaches

### Import packages

In [ ]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix



### TF-IDF vectorizer


<div>
<img src="https://www.kdnuggets.com/wp-content/uploads/awan_convert_text_documents_tfidf_matrix_tfidfvectorizer_3.png" width="590"/> 
</div>


Image sources: https://www.kdnuggets.com/2022/09/convert-text-documents-tfidf-matrix-tfidfvectorizer.html






##### Example on Small data

In [ ]:


# Sample data
documents = [
    "This is the first document.",
    "This document is the second document.",
    "And this is the third one.",
    "Is this the first document?"
]

In [ ]:
# Create a DataFrame for better visualization
df = pd.DataFrame({'Text': documents})

In [ ]:
# TF-IDF vectorization
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['Text'].tolist())

In [ ]:
# Convert the TF-IDF matrix to a DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())

In [ ]:
# Print the original data
print("Original Data:")
print(df)

In [ ]:
print(tfidf_matrix)

In [ ]:
# Print the TF-IDF matrix
print("\nTF-IDF Matrix:")
print(tfidf_df)

## Naive Bayes classifier trained on the TF-IDF features.

<div>
<img src="fig_bayes-nw.png" width="800"/> 
</div>


### Read data/Preparation

In [ ]:
# df = pd.read_csv("Womens_Clothing_E_Commerce_Reviews.csv")
df = pd.read_csv("imdb_reviews.csv")

In [ ]:
df.shape

In [ ]:
df.head(3)

In [ ]:
df['label'].unique()

In [ ]:
df['label'].unique()

In [ ]:
df.isna().sum()

### Split the dataset into training and testing sets

In [ ]:
train_data, test_data, train_labels, test_labels = train_test_split(df['text'], df['label'], test_size=0.3, random_state=42)

In [ ]:
print(train_data)

In [ ]:
print(train_labels)

### Create a pipeline

In [ ]:
sentiment_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('nb', MultinomialNB())
])

### Train the model using the pipeline

In [ ]:
sentiment_pipeline.fit(train_data, train_labels)


### Make predictions on the test set

In [ ]:
predictions = sentiment_pipeline.predict(test_data)

### Evaluate the model

In [ ]:

report = classification_report(test_labels, predictions)

print("Classification Report:\n", report)

In [ ]:
cm = confusion_matrix(test_labels, predictions)
cm

## Huggingface: Pre-trained sentiment analysis model

https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english

In [ ]:

from transformers import pipeline
sentiment_analyzer = pipeline('sentiment-analysis', model ="distilbert-base-uncased-finetuned-sst-2-english") #, revision ="af0f99b")
data = ["I love you", "I hate you"]
sentiment_analyzer(data)


In [ ]:
result = sentiment_analyzer("I love using this model!")
print(result)

## Huggingface: Thai 

### model="poom-sci/WangchanBERTa-finetuned-sentiment"

https://huggingface.co/poom-sci/WangchanBERTa-finetuned-sentiment

In [ ]:
from transformers import pipeline

sentiment_analyzer = pipeline('sentiment-analysis', model="poom-sci/WangchanBERTa-finetuned-sentiment")#, revision="b78d071")

data = ["อร่อยจัดๆ", "รอนานแท้"]
sentiment_analyzer(data)


In [ ]:
sentiment_analyzer("ข้าวบูด")

## Deploy on Streamlit Sharing

https://share.streamlit.io/  or https://huggingface.co/spaces

https://docs.streamlit.io/library/api-reference

https://github.com/



In [ ]:

%%writefile app_senti.py


import streamlit as st
from transformers import pipeline

# Load the sentiment analysis model
model_name = "poom-sci/WangchanBERTa-finetuned-sentiment"
sentiment_analyzer = pipeline('sentiment-analysis', model=model_name)

# Streamlit app
st.title("Thai Sentiment Analysis App")

# Input text
text_input = st.text_area("Enter Thai text for sentiment analysis", "ขอความเห็นหน่อย... ")

# Button to trigger analysis
if st.button("Analyze Sentiment"):
    # Analyze sentiment using the model
    results = sentiment_analyzer([text_input])

    # Extract sentiment and score
    sentiment = results[0]['label']
    score = results[0]['score']
    

    # Display result as progress bars
    st.subheader("Sentiment Analysis Result:")

    if sentiment == 'pos':
        st.success(f"Positive Sentiment (Score: {score:.2f})")
        st.progress(score)
    elif sentiment == 'neg':
        st.error(f"Negative Sentiment (Score: {score:.2f})")
        st.progress(score)
    else:
        st.warning(f"Neutral Sentiment (Score: {score:.2f})")
        st.progress(score)


In [ ]:
%%writefile requirements.txt
transformers
torch
